### 1. Load libraries

In [1]:
import numpy as np
import pandas as pd

In [2]:
from MTB_Score_ucsf.MTB_SCORE.Get_Stepdata import Get_Stepdata
from MTB_Score_ucsf.MTB_SCORE.Get_score_item import Get_score_items
from MTB_Score_ucsf.MTB_SCORE.Get_Scoresums import Get_Scoresum
from MTB_Score_ucsf.MTB_SCORE.Get_Scores import Get_Score
from MTB_Score_ucsf.MTB_SCORE.Combine import Combine
from MTB_Score_ucsf.MTB_SCORE.GetScore_SpellingAndVocab import GetScore_SpellingAndVocab


In [3]:
import synapseclient
from pyarrow import fs
import pyarrow.parquet as pq
import pandas as pd

#######################################################
# Change this to the synapse Id of the Parquet folder
PROJECT_ID = 'syn29357579'
#######################################################


### 2. Load dataset

In [4]:

syn = synapseclient.login()

token = syn.get_sts_storage_token(PROJECT_ID, permission="read_only") # , output_format='bash'
s3 = fs.S3FileSystem(access_key=token['accessKeyId'], 
                     secret_key = token['secretAccessKey'],
                     session_token = token['sessionToken'])

bucket_path = token['bucket']+'/'+token['baseKey']+'/'


#####
dataset_path1=bucket_path+'dataset_archivemetadata_v1/'

dataset1 = pq.ParquetDataset(dataset_path1, filesystem=s3)
dfid = dataset1.read().to_pandas()

#####
dataset_path2=bucket_path+'dataset_sharedschema_v1_steps/'

dataset2 = pq.ParquetDataset(dataset_path2, filesystem=s3)
stepdata= dataset2.read().to_pandas()

#####
dataset_path3=bucket_path+'dataset_sharedschema_v1/'

dataset3 = pq.ParquetDataset(dataset_path3, filesystem=s3)
task_data= dataset3.read().to_pandas()

Welcome, Vijay Yadav!




UPGRADE AVAILABLE

A more recent version of the Synapse Client (2.7.0) is available. Your version (2.6.0) can be upgraded by typing:
    pip install --upgrade synapseclient

Python Synapse Client version 2.7.0 release notes

https://python-docs.synapse.org/build/html/news.html



In [5]:
# dfid = pd.read_csv('loaded_data/dfid.csv', low_memory=False)
# stepdata = pd.read_csv('loaded_data/stepdata.csv', low_memory=False)
# task_data = pd.read_csv('loaded_data/task_data.csv', low_memory=False)

In [6]:
#Drop duplicates
dfid = dfid.drop_duplicates(subset=['recordid', 'startDate', 'assessmentid', 'taskRunUUID', 'appVersion', 
                                    'deviceInfo', 'endDate']).reset_index(drop=True)

### 3. Filter Scores

##### 3.1 Study membership details

In [7]:
demo_table_syn = syn.tableQuery('SELECT * FROM syn29300177') 
table_substudymembership = demo_table_syn.asDataFrame()
table_substudymembership = table_substudymembership.sort_values(by=['healthCode', 
                                                                    'participantVersion']).reset_index(drop=True)

table_substudymembership=table_substudymembership[['healthCode','studyMemberships']]
table_substudymembership=table_substudymembership.drop_duplicates(keep='last')

##### 3.2 Vocab

In [8]:
#vocab
vocab_score=GetScore_SpellingAndVocab(task_data,dfid,'Vocab')
vocab_score=pd.merge(vocab_score,table_substudymembership,how='inner',left_on='healthcode',right_on='healthCode')
del vocab_score['healthCode']

columns = ['vocab_THETA', 'vocab_SE']
vocab_score['score_dict'] = vocab_score[columns].to_dict(orient='records')
vocab_score = vocab_score.drop(columns=columns)


##### 3.3 Spelling

In [9]:
#spelling
spelling_score=GetScore_SpellingAndVocab(task_data,dfid,'Spelling')
spelling_score=pd.merge(spelling_score,table_substudymembership,how='inner',left_on='healthcode',right_on='healthCode')
del spelling_score['healthCode']

columns = ['spell_THETA', 'spell_SE']
spelling_score['score_dict'] = spelling_score[columns].to_dict(orient='records')
spelling_score = spelling_score.drop(columns=columns)


##### 3.4 PSM

In [10]:
#psm
PSM_score=GetScore_SpellingAndVocab(task_data,dfid,'PSM')
PSM_score=pd.merge(PSM_score,table_substudymembership,how='inner',left_on='healthcode',right_on='healthCode')
del PSM_score['healthCode']

columns = ['psm.adj.sum', 'psm.exact.sum']
PSM_score['score_dict'] = PSM_score[columns].to_dict(orient='records')
PSM_score = PSM_score.drop(columns=columns)

[WARNING] /home/vyadav/Vijay/MTB/scoring_pipeline/MTB_Score_ucsf/MTB_SCORE/GetScore_SpellingAndVocab.py:67: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  psm_taskData_scrd['psm.adj.sum'] = psm_taskData_scrd.loc[:,['scores_adjacentPairs1','scores_adjacentPairs2']].sum(axis=1)

[WARNING] /home/vyadav/Vijay/MTB/scoring_pipeline/MTB_Score_ucsf/MTB_SCORE/GetScore_SpellingAndVocab.py:68: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  psm_taskData_scrd['psm.exact.sum'] = psm_taskData_scrd.loc[:,['scores_exactMatch1

##### 3.5 MFS

In [11]:
#mfs
step_mfs=Get_Stepdata(stepdata,'MFS')
scoreitem_mfs=Get_score_items('MFS')
scoresum_mfs=Get_Scoresum(step_mfs,scoreitem_mfs,'MFS')

score_mfs=Get_Score(scoresum_mfs,task_data, dfid,'MFS')
score_mfs=pd.merge(score_mfs,table_substudymembership,how='inner',left_on='healthcode',right_on='healthCode')
del score_mfs['healthCode']

columns = ['mfs_sum']
score_mfs['score_dict'] = score_mfs[columns].to_dict(orient='records')
score_mfs = score_mfs.drop(columns=columns)

##### 3.6 DCCS

In [12]:
#dccs
step_dccs=Get_Stepdata(stepdata,'DCCS')
scoreitem_dccs=Get_score_items('DCCS')
scoresum_dccs=Get_Scoresum(step_dccs,scoreitem_dccs,'DCCS')
score_dccs=Get_Score(scoresum_dccs,task_data, dfid,'DCCS')
score_dccs=pd.merge(score_dccs,table_substudymembership,how='inner',left_on='healthcode',right_on='healthCode')
del score_dccs['healthCode']

columns = ['dccs_time', 'dccs_score', 'DCCS_accuracy']
score_dccs['score_dict'] = score_dccs[columns].to_dict(orient='records')
score_dccs = score_dccs.drop(columns=columns)

##### 3.7 Fname_b

In [13]:
#fname
step_fname=Get_Stepdata(stepdata,'FNAME')
scoreitem_fname=Get_score_items('FNAME')
scoresum_fname=Get_Scoresum(step_fname,scoreitem_fname,'FNAME')

score_fname=Get_Score(scoresum_fname,task_data, dfid,'FNAME')
score_fname=pd.merge(score_fname,table_substudymembership,how='inner',left_on='healthcode',right_on='healthCode')
del score_fname['healthCode']

columns = ['FNAME_theta', 'FNAME_sd']
score_fname['score_dict'] = score_fname[columns].to_dict(orient='records')
score_fname = score_fname.drop(columns=columns)

##### 3.8 Number Match

In [14]:
#nm
step_nm=Get_Stepdata(stepdata,'Number Match')
scoreitem_nm=Get_score_items('Number Match')
scoresum_nm=Get_Scoresum(step_nm,scoreitem_nm,'Number Match')

score_nm=Get_Score(scoresum_nm,task_data, dfid,'Number Match')
score_nm=pd.merge(score_nm,table_substudymembership,how='inner',left_on='healthcode',right_on='healthCode')
del score_nm['healthCode']

columns = ['numberMatch_raw']
score_nm['score_dict'] = score_nm[columns].to_dict(orient='records')
score_nm = score_nm.drop(columns=columns)

##### 3.9 Flanker

In [15]:
#flanker
step_flanker=Get_Stepdata(stepdata,'Flanker')
scoreitem_flanker=Get_score_items('Flanker')
scoresum_flanker=Get_Scoresum(step_flanker,scoreitem_flanker,'Flanker')

score_flanker=Get_Score(scoresum_flanker,task_data, dfid,'Flanker')
score_flanker=pd.merge(score_flanker,table_substudymembership,how='inner',left_on='healthcode',right_on='healthCode')
del score_flanker['healthCode']

columns = ['flkr_time', 'flkr_score', 'FLANKER_accuracy']
score_flanker['score_dict'] = score_flanker[columns].to_dict(orient='records')
score_flanker = score_flanker.drop(columns=columns)

### 4. Combine Score

In [16]:
stack_merged, scores_merged = Combine(score_mfs, score_dccs, score_fname, score_nm, score_flanker, PSM_score, 
                                      spelling_score, vocab_score, dfid)


In [17]:
#stack_merged.to_csv('MTB_UCSF_scores.csv', index=False)